<a id="title_ID"></a>
# JWST Pipeline Validation Testing Notebook: MIRI MRS Spec2: Assign_wcs()
<span style="color:red"> **Instruments Affected**</span>: FGS, MIRI, NIRCam, NIRISS, NIRSpec 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#JWST-Pipeline-Validation-Testing-Notebook:-MIRI-MRS" data-toc-modified-id="JWST-Pipeline-Validation-Testing-Notebook:-MIRI-MRS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>JWST Pipeline Validation Testing Notebook: MIRI MRS</a></span><ul class="toc-item"><li><span><a href="#Spec2:-Assign_wcs()" data-toc-modified-id="Spec2:-Assign_wcs()-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Spec2: Assign_wcs()</a></span><ul class="toc-item"><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Table of Contents</a></span></li></ul></li></ul></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Calibration-WG-Requested-Algorithm:" data-toc-modified-id="Calibration-WG-Requested-Algorithm:-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Calibration WG Requested Algorithm:</a></span></li></ul></li></ul></li><li><span><a href="#Loading-Data" data-toc-modified-id="Loading-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Loading Data</a></span></li><li><span><a href="#Run-JWST-Pipeline---AssignWCS" data-toc-modified-id="Run-JWST-Pipeline---AssignWCS-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Run JWST Pipeline - AssignWCS</a></span><ul class="toc-item"><li><span><a href="#Output-checks-and-tests" data-toc-modified-id="Output-checks-and-tests-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Output checks and tests</a></span></li><li><span><a href="#About-this-Notebook" data-toc-modified-id="About-this-Notebook-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>About this Notebook</a></span></li></ul></li></ul></div>

<a id="imports_ID"></a>
# Imports
List the library imports and why they are relevant to this notebook.

* os, glob for general  OS operations
* numpy
* astropy.io for opening fits files
* astropy.utils for remote data retrieval
* IPython.display for printing markdown output
* jwst.datamodels for building model for JWST Pipeline
* jwst.module.PipelineStep is the pipeline step being tested
* miricoord for coordinate conversion tools

[Top of Page](#title_ID)

In [1]:
import glob
import numpy as np
from numpy.testing import assert_allclose
import os

import astropy.io.fits as fits
import astropy.io.ascii as ascii
from ci_watson.artifactory_helpers import get_bigdata
from IPython.display import Markdown

from jwst.assign_wcs import AssignWcsStep
import miricoord.mrs.toolversions.mrs_pipetools_cdp8b as tv
from miricoord.mrs import mrs_tools as mrst

%matplotlib inline

In [2]:
use_tempdir = True

# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
import shutil

if use_tempdir:
    data_dir = TemporaryDirectory()

    # If you have files that are in the notebook's directory, but that the notebook will need to use while
    # running, copy them into the temporary directory here.
    #
    # files = ['name_of_file']
    # for file_name in files:
    #     shutil.copy(file_name, os.path.join(data_dir.name, file_name))

    # Save original directory
    orig_dir = os.getcwd()

    # Move to new directory
    os.chdir(data_dir.name)

# For info, print out where the script is running
print("Running in {}".format(os.getcwd()))


Running in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpq08ajclq


<a id="intro_ID"></a>
# Introduction


In this notebook we will test the assign_wcs step of Spec2Pipeline() for **MRS** observations.

Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/index.html#assign-wcs-step

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/assign_wcs


### Calibration WG Requested Algorithm: 

https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Spectral+GWCS+Information



[Top of Page](#title_ID)

<a id="data_ID"></a>
# Loading Data

We are using simulated MRS observations, generated with MIRISim v2.3.0 (as of Dec 2020). ***(Needs updating)*** 

[Top of Page](#title_ID)

In [3]:
mock12A = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock12A-0.fits')

mock12B = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock12B-0.fits')

mock12C = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock12C-0.fits')

mock34A = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock34A-0.fits')

mock34B = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock34B-0.fits')

mock34C = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock34C-0.fits')

files = [mock12A, mock12B, mock12C, mock34A, mock34B, mock34C]

Collect configuration files locally if they aren't yet there.

In [4]:
# if not os.path.exists('../cfg_files/'):
#     os.mkdir('../cfg_files/')
#     cfgs = collect_pipeline_cfgs.collect_pipeline_cfgs(dst='../cfg_files/')

<a id="pipeline_ID"></a>
# Run JWST Pipeline - AssignWCS

For this test we do not need to run the data through the Detector1() pipeline (in fact, this data will fail in Detector1). We begin by running the data through AssignWCS in spec2 using the 'assign_wcs.cfg' file. The AssignWcs step should not change anything in the science data or error extension. It attaches the WCS information to the data, so that a location in detector pixels can be reconstructed to an RA, dec, wavelength in spatial and spectral space.

[Top of Page](#title_ID)

In [5]:
awcs_out = []

for dd in files: #det1_out:
    awcs = AssignWcsStep.call(dd, save_results=True)
    awcs_out.append(awcs)

2021-07-02 13:15:41,982 - stpipe.AssignWcsStep - INFO - AssignWcsStep instance created.


2021-07-02 13:15:42,078 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep running with args ('/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpq08ajclq/mock12A-0.fits',).


2021-07-02 13:15:42,080 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'sip_approx': True, 'sip_max_pix_error': 0.25, 'sip_degree': None, 'sip_max_inv_pix_error': 0.25, 'sip_inv_degree': None, 'sip_npoints': 32, 'slit_y_low': -0.55, 'slit_y_high': 0.55}


2021-07-02 13:15:46,185 - stpipe.AssignWcsStep - INFO - Created a MIRI mir_mrs pipeline with references {'distortion': '/grp/crds/cache/references/jwst/jwst_miri_distortion_0031.asdf', 'filteroffset': None, 'specwcs': '/grp/crds/cache/references/jwst/jwst_miri_specwcs_0024.asdf', 'regions': '/grp/crds/cache/references/jwst/jwst_miri_regions_0027.asdf', 'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_miri_wavelengthrange_0005.asdf', 'camera': None, 'collimator': None, 'disperser': None, 'fore': None, 'fpa': None, 'msa': None, 'ote': None, 'ifupost': None, 'ifufore': None, 'ifuslicer': None}


2021-07-02 13:15:48,213 - stpipe.AssignWcsStep - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.8/site-packages/gwcs/geometry.py:203: RuntimeWarning: invalid value encountered in remainder
  lon = np.mod(lon, 360.0 * u.deg if nquant else 360.0)



...

2021-07-02 13:16:28,453 - stpipe.AssignWcsStep - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.8/site-packages/gwcs/geometry.py:203: RuntimeWarning: invalid value encountered in remainder
  lon = np.mod(lon, 360.0 * u.deg if nquant else 360.0)



2021-07-02 13:16:28,538 - stpipe.AssignWcsStep - INFO - Update S_REGION to POLYGON ICRS  44.998721627 -0.001305713 45.000866457 -0.001305713 45.000866457 0.000719789 44.998721627 0.000719789


2021-07-02 13:16:28,540 - stpipe.AssignWcsStep - INFO - COMPLETED assign_wcs


2021-07-02 13:16:29,063 - stpipe.AssignWcsStep - INFO - Saved model in mock34C-0_assignwcsstep.fits


2021-07-02 13:16:29,064 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep done


<a id="residual_ID"></a>
## Output checks and tests

We can use the output model to run the following checks:

* the SCI extension is unchanged from the step input (accept default tolerances for the np.allclose() function)
* we use the wcs information assigned in the step to check the RA, dec, wavelength assignment at some reference locations. 


[Top of Page](#title_ID)

In [6]:
# Do a numeric check between input and output
for (aa, dd) in zip(awcs_out, files):
    data = fits.getdata(dd)
    filename = os.path.split(dd)[-1]
    assert np.allclose(aa.data, data, equal_nan=True), "Step input and output don't match for {}".format(filename)

Load the set of test coordinates from ``miricoord``. Then calculate the wavelength of these coordinates from the assign_wcs output models, and verify that they are close. If all the ``assert`` tests pass, this test is successful and complete.

In [7]:
def verify_coords(imgnum, ch, ref_data):
    """Uses predefined test cooridnates to check WCS output."""
    
    xt = ref_data[ch]['x']
    yt = ref_data[ch]['y']
    v2t = ref_data[ch]['v2']
    v3t = ref_data[ch]['v3']
    lamt = ref_data[ch]['lam']
    
    lamtest1 = np.zeros(2)
    lamtest2 = np.zeros(2)
    atest1 = np.zeros(2)
    btest1 = np.zeros(2)
    v2test1 = np.zeros(2)
    v3test1 = np.zeros(2)
    lt1 = np.zeros(2)
    
    print('**** TEST SET {}****'.format(ch))
    print(' x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test')
    for i in range (2):
        values = mrst.xytoabl(xt[i], yt[i], ch)
        atest1[i], btest1[i], lt1[i] = values['alpha'], values['beta'], values['lam']
        v2test1[i], v3test1[i] = mrst.abtov2v3(atest1[i], btest1[i], ch)
        ra, dec, lamtest1[i] = awcs_out[imgnum].meta.wcs(xt[i], yt[i])
        print('{0}  {1}   {2:.5f}    {3:.5f}    {4:.5f}   {5:.5f}   {6:.5f}   {7:.5f}'.format(
            xt[i], yt[i], lamt[i], lamtest1[i], v2t[i], v3t[i], v2test1[i], v3test1[i]))

        assert_allclose(lamt[i], lamtest1[i], atol=0.05)
        assert_allclose(v2t[i], v2test1[i], atol=0.05)
        assert_allclose(v3t[i], v3test1[i], atol=0.05)
        
    print('\n')

In [8]:
mrd = tv.mrs_ref_data
chan = {'0':'1A',
        '1':'1B',
        '2':'1C',
        '3':'2A',
        '4':'2B',
        '5':'2C',
        '6':'3A',
        '7':'3B',
        '8':'3C',
        '9':'4A',
        '10':'4B',
        '11':'4C'}

for i in range(12):
    ch = chan[str(i)]
    
    if i < 6 and 'A' in ch:
        imgnum = 0
    elif i < 6 and 'B' in ch:
        imgnum = 1
    elif i < 6 and 'C' in ch:
        imgnum = 2
    elif i >= 6 and 'A' in ch:
        imgnum = 3
    elif i >= 6 and 'B' in ch:
        imgnum = 4
    elif i >= 6 and 'C' in ch:
        imgnum = 5
        
    verify_coords(imgnum, ch, mrd)

**** TEST SET 1A****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
76.0  512.0   5.34855    5.34855    -503.57285   -318.57499   -503.57285   -318.57499
354.0  700.0   5.51364    5.51364    -503.49798   -317.50901   -503.49798   -317.50901


**** TEST SET 1B****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
76.0  512.0   6.16831    6.16831    -503.64310   -318.72773   -503.64310   -318.72773
355.0  700.0   6.35801    6.35801    -503.37070   -317.69382   -503.37070   -317.69382


**** TEST SET 1C****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test


78.0  512.0   7.00661    7.00661    -503.55984   -318.43677   -503.55984   -318.43677
356.0  700.0   7.21846    7.21846    -503.45976   -317.37795   -503.45976   -317.37795


**** TEST SET 2A****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
574.0  512.0   8.13946    8.13946    -503.65782   -319.37098   -503.65782   -319.37098
719.0  700.0   8.42388    8.42388    -503.39070   -317.71319   -503.39070   -317.71319


**** TEST SET 2B****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
570.0  512.0   9.49092    9.49092    -503.87244   -319.60662   -503.87244   -319.60662
715.0  700.0   9.82611    9.82611    -503.58468   -317.95262   -503.58468   -317.95262


**** TEST SET 2C****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test


573.0  512.0   10.90956    10.90956    -503.70624   -319.54349   -503.70624   -319.54349
718.0  700.0   11.29266    11.29266    -503.42920   -317.88865   -503.42920   -317.88865


**** TEST SET 3A****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
918.0  512.0   12.58609    12.58609    -504.57532   -319.35962   -504.57532   -319.35962
827.0  700.0   12.17180    12.17180    -504.34284   -317.03633   -504.34284   -317.03633


**** TEST SET 3B****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
919.0  512.0   14.60074    14.60074    -504.29129   -319.59777   -504.29129   -319.59777


827.0  700.0   14.12035    14.12035    -503.81514   -317.30170   -503.81514   -317.30170


**** TEST SET 3C****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
917.0  512.0   16.86062    16.86062    -504.29179   -319.58642   -504.29179   -319.58642
826.0  700.0   16.30565    16.30565    -504.06099   -317.26146   -504.06099   -317.26146


**** TEST SET 4A****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
195.0  512.0   19.42967    19.42967    -503.73916   -321.71985   -503.73916   -321.71985
232.0  700.0   18.73379    18.73379    -502.92871   -317.85961   -502.92871   -317.85961


**** TEST SET 4B****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
192.0  512.0   22.47574    22.47574    -503.70510   -321.66373   -503.70510   -321.66373
229.0  700.0   21.67075    21.67075    -502.98915   -317.78403   -502.98915   -317.78403


**** TEST SET 4C****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3te

194.0  512.0   26.29238    26.29238    -503.71719   -321.57006   -503.71719   -321.57006
231.0  700.0   25.35069    25.35069    -502.92825   -317.72523   -502.92825   -317.72523




<a id="about_ID"></a>
## About this Notebook
**Author:** Mike Engesser, Science Support Analyst, MIRI branch 
<br>(Adapted from Sarah Kendrew's AssignWCS LRS-SLIT notebook.)
<br>**Updated On:** 4/01/2021

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 